In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By  # 导入By类
from selenium.webdriver.common.keys import Keys # 导入Key类
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# 定义处理弹窗的函数
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

In [ ]:
# 配置浏览器
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Selenium\AutomationProfile"
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Program Files\ChromeDir"
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9998")
options.add_argument("--disable-javascript")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(60)

In [ ]:
page_source = driver.page_source

In [ ]:
#
def execute_initial_scripts():
        driver.execute_script("""
            // 移除右键菜单禁用
            document.oncontextmenu = null;

            // 移除所有键盘事件监听器
            document.onkeydown = null;
            document.onkeypress = null;
            document.onkeyup = null;

            // 覆盖按键事件，防止 F12 被拦截
            window.addEventListener('keydown', function(event) {
                if (event.key === 'F12' || (event.key === 'I' && event.ctrlKey && event.shiftKey)) {
                    event.stopImmediatePropagation();
                }
            }, true);

            // 重写 console.log 方法来调试
            console.log = function(message) {
                var console = document.createElement('div');
                console.style.background = 'yellow';
                console.style.padding = '10px';
                console.style.margin = '10px';
                console.style.zIndex = '9999';
                console.style.position = 'absolute';
                console.innerHTML = message;
                document.body.appendChild(console);
            };

            // 获取视频元素并覆盖暂停方法
            var video = document.querySelector('video');
            if (video) {
                video.pause = function() {
                    console.log('视频暂停被拦截');
                };
                video.play();
            } else {
                console.log('未找到视频元素');
            }
        """)
# 禁止自动暂停并播放
def pause_and_play():
        driver.execute_script("""
        var video = document.querySelector('video');
        if (video.paused) {
            video.play();
            console.log("视频恢复播放");
        }
    """)

# 定义检查视频播放状态并切换到下一个视频的函数
def switch_video(current_video_index, video_elements):
    # 检查播放按钮状态
    next_video_index = (current_video_index + 1) % len(video_elements)
    video_elements[next_video_index].click()
    time.sleep(3)
    pause_and_play()
    return next_video_index
# 定义获取视频总时长的函数
def get_video_duration(max_attempts=30, delay=2):
    for attempt in range(max_attempts):
        try:
            duration_element = driver.find_element(By.XPATH, "//div[@class='vjs-duration vjs-time-control vjs-control']//span[@class='vjs-duration-display']")
            duration_text = duration_element.text
            if duration_text and duration_text != "0:00":
                print(f"当前视频的总时长: {duration_text}")
                pause_and_play()
                return duration_text
        except Exception as e:
            print(f"尝试获取视频时长 (尝试 {attempt + 1}/{max_attempts}): {e}")
        
        time.sleep(delay)
        pause_and_play()
    
    print("无法获取有效的视频时长")
    return None
# 点击弹出的窗口
def handle_popup():
    try:
        # 使用更精确的选择器来查找弹窗的确定按钮
        confirm_button = driver.find_element(By.XPATH, "//div[@class='el-dialog__footer']//button[@class='el-button el-button--primary']")
        confirm_button.click()
        pause_and_play()
        print("Popup confirmed")
    except NoSuchElementException:
        pause_and_play()
        print("Popup not found")
    except TimeoutException:
        pause_and_play()
        print("Popup handling timed out")
    except Exception as e:
        pause_and_play()
        print("An error occurred while handling the popup:", e)
# 时间转换
def time_to_seconds(time_str):
    parts = time_str.split(':')
    if len(parts) == 2:
        return int(parts[0]) * 60 + int(parts[1])
    elif len(parts) == 3:
        return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
    else:
        raise ValueError("Invalid time format")

In [ ]:

def play_video_playlist(current_video_index):
    # 获取视频列表
    video_elements = driver.find_elements(By.XPATH, "//div[@class='el-step__title is-wait']")
    current_video_index = current_video_index

    while True:
        # 点击并播放当前视频
        video_elements[current_video_index].click()
        time.sleep(3)
        pause_and_play()
        # 获取视频时长
        duration = None
        while duration is None:
            duration = get_video_duration()
            if duration is None:
                print("重试获取视频时长...")
                time.sleep(5)
        duration = get_video_duration()
        if duration:
            video_length = time_to_seconds(duration)
            max_play_time = video_length + 300  # 视频长度加5分钟
        else:
            print("无法获取视频时长。")
            max_play_time = 2400  # 设视频长度为40分钟
        start_time = time.time()
        last_popup_check = start_time

        while time.time() - start_time < max_play_time:
            # 每100秒检查一次弹窗
            if time.time() - last_popup_check >= 100:
                handle_popup()
                last_popup_check = time.time()

            pause_and_play()
            time.sleep(10)  # 等待10秒再次检查

        # 切换到下一个视频
        current_video_index = (current_video_index + 1) % len(video_elements)

In [ ]:
# 初始化
execute_initial_scripts()

In [ ]:
# 执行自动播放

play_video_playlist(8)